## import packages and define cnn & helper functions

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as nnF
from torch.utils.data import DataLoader as torch_dataloader
from torch.utils.data import Dataset as torch_dataset
import torch.optim as optim

In [279]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=256, 
                               kernel_size=3, stride=2, padding=2)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, 
                               kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=256, 
                               kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv1d(in_channels=256, out_channels=256, 
                               kernel_size=3, stride=2, padding=2)
        self.conv5 = nn.Conv1d(in_channels=256, out_channels=1, 
                               kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(2, out_features=1024)
        self.fc2 = nn.Linear(1024, 2)
        self.norm3 = nn.BatchNorm1d(num_features=256) # 1 channel
        self.norm4 = nn.BatchNorm1d(num_features=256)
        self.norm5 = nn.BatchNorm1d(num_features=256)
        self.avg1 = nn.AvgPool1d(kernel_size=2, stride=2,padding=1)
        self.avg2 = nn.AvgPool1d(kernel_size=2, stride=2,padding=1)
        self.avg3 = nn.AvgPool1d(kernel_size=2, stride=2,padding=1)
    def forward(self, x):
        x=nnF.relu(self.conv1(x))
        x1 = self.avg1(x)
        # print(nnF.relu(self.conv2(x)).shape,x1.shape)
        x=nnF.relu(self.conv2(x)) + x1 # residual connection
        x1 = self.avg2(x)
        # print(nnF.relu(self.conv3(self.norm3(x))).shape,x1.shape)
        x=nnF.relu(self.conv3(self.norm3(x))) + x1
        x1 = self.avg3(x)
#         print(nnF.relu(self.conv4(x)).shape,x1.shape)
        x=nnF.relu(self.conv4(self.norm4(x))) + x1
        # print(nnF.relu(self.conv5(self.norm5(x))).shape)
        x=nnF.relu(self.conv5(self.norm5(x))).squeeze(1)
        # print(x.shape)
        x=nnF.relu(self.fc1(x))
        z=self.fc2(x)
        #y=nnF.softmax(z, dim=1)
        return z

In [118]:
def cal_accuracy(confusion):
    #input: confusion is the confusion matrix
    #output: acc is the standard classification accuracy
    M=confusion.copy().astype('float32')
    acc = M.diagonal().sum()/M.sum()    
    sens=np.zeros(M.shape[0])
    prec=np.zeros(M.shape[0]) 
    for n in range(0, M.shape[0]):
        TP=M[n,n]
        FN=np.sum(M[n,:])-TP
        FP=np.sum(M[:,n])-TP
        sens[n]=TP/(TP+FN)
        prec[n]=TP/(TP+FP)       
    return acc, sens, prec

In [ ]:
def train(model, device, optimizer, dataloader, epoch):    
    model.train()#set model to training mode
    loss_train=0
    acc_train =0 
    sample_count=0
    for batch_idx, (X, Y) in enumerate(dataloader):
#         print(X.shape)
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()#clear grad of each parameter
        Z = model(X)#forward pass
#         print(Z.shape, Y.shape)
        loss = nnF.cross_entropy(Z, Y)
        loss.backward()#backward pass
        optimizer.step()#update parameters
        loss_train+=loss.item()
        #do not need softmax
        Yp = Z.data.max(dim=1)[1]  # get the index of the max               
        acc_train+= torch.sum(Yp==Y).item()
        sample_count+=X.size(0)
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{:.0f}%]\tLoss: {:.6f}'.format(
                    epoch, 100. * batch_idx / len(dataloader), loss.item()))
    loss_train/=len(dataloader)
    #acc_train/=len(dataloader.dataset) 
    acc_train/=sample_count    
    return loss_train, acc_train

In [ ]:
def test(model, device, dataloader):
    Ys, Yps = [],[]
    model.eval()#set model to evaluation mode
    acc_test =0
    confusion=np.zeros((2,2))
    with torch.no_grad(): # tell Pytorch not to build graph in the with section
        for batch_idx, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
#             print(X.shape)
            Z = model(X)#forward pass
            #do not need softmax
            Yp = Z.data.max(dim=1)[1]  # get the index of the max 
            Ys.extend(Y)
            Yps.extend(Yp)
            acc_test+= torch.sum(Yp==Y).item()
            for i in range(0, 2):
                for j in range(0, 2):
                    confusion[i,j]+=torch.sum((Y==i)&(Yp==j)).item()
    acc, sens, prec=cal_accuracy(confusion)
    return acc, (confusion, sens, prec), Ys, Yps

## preprocess data

In [ ]:
import pandas as pd
data = pd.read_csv("pima-indians-diabetes.csv")

In [ ]:
cols_to_include = "preg	plasma	pressure	skin	insulin	bmi	pedigree	age".split()
X = data[cols_to_include].values
Y = data["class"].values

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.utils.class_weight import compute_class_weight
# scaler = MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)

# class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
# class_weights = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
len(cols_to_include)

In [ ]:
X.shape,Y.shape

In [ ]:
class MyDataset(torch_dataset):
    def __init__(self, X, Y):
        self.X=X
        self.Y=Y
    def __len__(self):
        #return the number of data points
        return self.X.shape[0]
    def __getitem__(self, idx):        
        #we can use DatasetName[idx] to get a data point (x,y) with index idx
        x=torch.tensor(self.X[idx], dtype=torch.float32)
        y=torch.tensor(self.Y[idx], dtype=torch.int64)
        x=x.reshape(1,-1)
        return x, y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)
dataset_train=MyDataset(X_train, Y_train)
dataset_val=MyDataset(X_val, Y_val)

In [ ]:
loader_train = torch_dataloader(dataset_train, batch_size=4, shuffle=True, num_workers=0)
loader_val = torch_dataloader(dataset_val, batch_size=4, shuffle=False, num_workers=0) 

## initialize model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=Net()
model.to(device)

In [ ]:
optimizer = optim.Adamax(model.parameters(), lr=0.0001, weight_decay=1e-4)
loss_train_list=[]
acc_train_list=[]
acc_val_list=[]
epoch_save=-1

## train

In [293]:
best_acc_val = -9999
for epoch in range(epoch_save+1, 500):
    #-------- training --------------------------------
    loss_train, acc_train =train(model, device, optimizer, loader_train, epoch)    
    loss_train_list.append(loss_train)
    acc_train_list.append(acc_train)
    print('epoch', epoch, 'training loss:', loss_train, 'acc:', acc_train)
    #-------- validation --------------------------------
    acc_val, other_val, _, __ = test(model, device, loader_val)
    acc_val_list.append(acc_val)
    print('epoch', epoch, 'validation acc:', acc_val)
    #--------save model-------------------------
    result = (loss_train_list, acc_train_list, 
              acc_val_list, other_val)
    if acc_val > best_acc_val:
        best_acc_val = acc_val
        torch.save(model.state_dict(), 'best_model_6.pth')

Train Epoch: 0 [0%]	Loss: 0.690365
Train Epoch: 0 [93%]	Loss: 0.234365
epoch 0 training loss: 0.4153240993963899 acc: 0.8158508158508159
epoch 0 validation acc: 0.6111111
Train Epoch: 1 [0%]	Loss: 0.174468
Train Epoch: 1 [93%]	Loss: 0.626032
epoch 1 training loss: 0.413716421669556 acc: 0.8041958041958042
epoch 1 validation acc: 0.6203704
Train Epoch: 2 [0%]	Loss: 0.660497
Train Epoch: 2 [93%]	Loss: 0.369900
epoch 2 training loss: 0.3882978275694229 acc: 0.8228438228438228
epoch 2 validation acc: 0.6018519
Train Epoch: 3 [0%]	Loss: 0.304734
Train Epoch: 3 [93%]	Loss: 0.112679
epoch 3 training loss: 0.39989553698924957 acc: 0.8181818181818182
epoch 3 validation acc: 0.5092593
Train Epoch: 4 [0%]	Loss: 0.094074
Train Epoch: 4 [93%]	Loss: 0.200050
epoch 4 training loss: 0.39446713582233145 acc: 0.8205128205128205
epoch 4 validation acc: 0.7222222
Train Epoch: 5 [0%]	Loss: 0.563573
Train Epoch: 5 [93%]	Loss: 0.296456
epoch 5 training loss: 0.41487687901179826 acc: 0.8181818181818182
epoch 

In [295]:
max(acc_val_list)

0.7962963

In [296]:
best_epoch=np.argmax(acc_val_list)
best_epoch

34

In [298]:
# previous model was better
model=Net()
model.load_state_dict(torch.load('best_model_4.pth'))
model.to(device)

Net(
  (conv1): Conv1d(1, 256, kernel_size=(3,), stride=(2,), padding=(2,))
  (conv2): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv4): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(2,))
  (conv5): Conv1d(256, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=2, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
  (norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avg1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
  (avg2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
  (avg3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
)

## validation set evaluation

In [299]:
acc, (confusion, recall, prec), Ys, Yps = test(model, device, loader_val)
print('Accuracy (average)', acc)
print('Recall', recall)
print('Precision', prec)
print('Confusion \n', confusion)

from sklearn.metrics import f1_score
f1 = f1_score(Ys, Yps)

print("F1 Score:", f1)
'''
Accuracy (average) 0.8425926
Recall [0.875      0.77777779]
Precision [0.88732392 0.75675678]
Confusion 
 [[63.  9.]
 [ 8. 28.]]
F1 Score: 0.7671232876712328
'''

Accuracy (average) 0.8425926
Recall [0.875      0.77777779]
Precision [0.88732392 0.75675678]
Confusion 
 [[63.  9.]
 [ 8. 28.]]
F1 Score: 0.7671232876712328


'\nAccuracy (average) 0.8425926\nRecall [0.875      0.77777779]\nPrecision [0.88732392 0.75675678]\nConfusion \n [[63.  9.]\n [ 8. 28.]]\nF1 Score: 0.7671232876712328\n'

## test f1 score will likely be close to .767

In [300]:
model=Net()
model.load_state_dict(torch.load('best_model_4.pth'))
model.to(device)

Net(
  (conv1): Conv1d(1, 256, kernel_size=(3,), stride=(2,), padding=(2,))
  (conv2): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv4): Conv1d(256, 256, kernel_size=(3,), stride=(2,), padding=(2,))
  (conv5): Conv1d(256, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=2, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
  (norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avg1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
  (avg2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
  (avg3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(1,))
)

In [301]:
eval = pd.read_csv("pima-indians-diabetes_eval_no_class.csv")
eval.head(1)

,index,preg,plasma,pressure,skin,insulin,bmi,pedigree,age
0,171,6,134,70,23,130,35.4,0.542,29


## blind test diabetes predictions

In [204]:
X_test = eval[cols_to_include].values
model.eval()
Z_pred = []
with torch.no_grad():
    for i in range(len(X_test)):
        X = X_test[4*i:4*(i+1)]
        X = torch.tensor(X, dtype=torch.float32).unsqueeze(1) # add channel dim
        # print(np.array(torch.argmax((nnF.softmax(model(X), dim=1)),dim=1)))
        Z = np.array(torch.argmax((nnF.softmax(model(X), dim=1)),dim=1))
        Z_pred.extend(list(Z))

In [205]:
Z_pred

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1]

In [208]:
len(eval),len(Z_pred)

(231, 231)

In [206]:
eval["class"] = Z_pred

In [207]:
eval

,index,preg,plasma,pressure,skin,insulin,bmi,pedigree,age,class
0,171,6,134,70,23,130,35.4,0.542,29,0
1,696,3,169,74,19,125,29.9,0.268,31,1
2,442,4,117,64,27,120,33.2,0.230,24,0
3,638,7,97,76,32,91,40.9,0.871,32,0
4,393,4,116,72,12,87,22.1,0.463,37,0
...,...,...,...,...,...,...,...,...,...,...
226,257,2,114,68,22,0,28.7,0.092,25,0
227,206,8,196,76,29,280,37.5,0.605,57,1
228,390,1,100,66,29,196,32.0,0.444,42,0
229,487,0,173,78,32,265,46.5,1.159,58,1


## save to file

In [209]:
eval.to_csv("predictions.csv", index=False)